In [ ]:
import sys

sys.path.insert(0, '../03_Remake_with_TDD/algorithms')
from ht_time import *

sys.path.insert(0, '../03_Remake_with_TDD/postgresql')
from ht_player_postgresql import *

In [ ]:
HattrickTime = HattrickTime()
folder = HattrickTime.findLastSunOrWed()
# folder = ht_time.findLastSunOrWed('2018/04/04')
# folder = '2017/11/01'
print("folder =", folder)

In [ ]:
folder = '2018/08/05'
print("folder =", folder)

In [ ]:
from datetime import date

nowDate = date.today().strftime('%Y/%m/%d')
nowDate

In [ ]:
import sys

sys.path.insert(0, '../97_Parse/')
from parseSkillTable import *

sys.path.insert(0, '../98_Crawler/01_DualMonitor/')
from DualMonitor import getIsDualMonitor

sys.path.insert(0, '../98_Crawler/02_WebBrowser/')
from FirefoxTab import clickTabOfFireFox

sys.path.insert(0, '../98_Crawler/03_OnScreen/')
from Empty import clickEmpty
from MoveScroll import moveScroll
from MoveFindMoveClick import moveByImage

sys.path.insert(0, '../98_Crawler/04_Login/')
from HattrickLogInOut import clickHattrickLogin, clickHattrickLogout

In [ ]:
# isDualMonitor
isDualMonitor = getIsDualMonitor()

# Hattrick HeavyDefence Tab
clickTabOfFireFox(4, isDualMonitor)

# # Hattrick Login by Type
clickEmpty(305, isDualMonitor)
moveScroll(20)
clickHattrickLogin(500, 305, isDualMonitor)

In [ ]:
# isDualMonitor
isDualMonitor = getIsDualMonitor()

# Hattrick HeavyDefence Tab
clickTabOfFireFox(4, isDualMonitor)

# # Hattrick Login by Type
clickEmpty(305, isDualMonitor)
moveScroll(20)
clickHattrickLogin(500, 305, isDualMonitor)

# Hattrick MyClub
moveScroll(20)
moveByImage(530, 342, 77, 42, isDualMonitor, 'screen-MyClub.png')

# Hattrick Player
clickEmpty(700, isDualMonitor)
moveScroll(20)
moveByImage(387, 700, 72, 59, isDualMonitor, 'screen-Player.png')

# Hattrick SkillTable
clickEmpty(593, isDualMonitor)
moveScroll(20)
moveByImage(546, 595, 137, 52, isDualMonitor, 'screen-SkillTable.png')

# Hattrick Since
moveScroll(-7)
moveByImage(584, 943, 168, 25, isDualMonitor, 'screen-Since.png')

# Hattrick Copy
clickEmpty(236, isDualMonitor)
moveScroll(20)
moveByImage(1075, 635, 32, 25, isDualMonitor, 'screen-Copy.png')

# Hattrick Logout
moveScroll(20)
clickHattrickLogout(1270, 370, isDualMonitor)

# Jupyter Notebook Tab
clickTabOfFireFox(7, isDualMonitor)

In [ ]:
import pyperclip # The name you have the file

tmpClip = pyperclip.paste()
print(tmpClip)

In [ ]:
from ht_file import *

In [ ]:
HattrickFile = HattrickFile()
HattrickFile.check_and_create_folder(folder)

In [ ]:
HattrickFile.check_and_delete_file(folder + '/player.html')

In [ ]:
with open(folder + '/player.html', 'w') as f:
    f.write(tmpClip)

In [ ]:
import csv
lineList = [] # emptyList

with open(folder + '/player.html', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\n')
    for row in reader:
        #print(row[0])
        lineList.append(row[0])
lineList

In [ ]:
#lineList = tmpClip.split("\n");

lineList = removeHattrickTag(lineList)
lineList = combineTwoLine(lineList)
lineList = modifySharpToNumber(lineList)
lineList = modifySpToSpecial(lineList)
lineList = removeFirstColumn(lineList)
lineList = dividePlayerAndPlayerID(lineList)
lineList = modifySince(lineList, folder, nowDate)
lineList = modifyMB(lineList)
lineList = modifyLast(lineList)
lineList = assignEmptyForTC(lineList)
lineList = assignEmptyForPH(lineList)
lineList = modifyKPPos(lineList)
lineList = modifyWBPos(lineList)
lineList = modifyCDPos(lineList)
lineList = modifyWPos(lineList)
lineList = modifyIMPos(lineList)
lineList = modifyFWPos(lineList)
lineList = removeLastString(lineList)

In [ ]:
# for debugging
# for i in range(0, len(lineList)):
#     print(i, lineList[i], sep='\t')

In [ ]:
print('Player Total Count =', len(lineList)-1)
for line in lineList:
    col = line.split(",")
    print(len(col), col[0], sep="\t" ) # 36
    #print(line)

In [ ]:
HattrickFile.check_and_delete_file(folder + '/player.html')
HattrickFile.check_and_create_folder(folder)
HattrickFile.check_and_delete_file(folder + '/player.txt')

In [ ]:
with open(folder+'/player.txt', 'w') as f:
    for line in lineList:
        f.write(line + "\n")

In [ ]:
# player_skill_table from file to table

In [ ]:
#import pymsgbox

In [ ]:
#pymsgbox.alert(text='', title='', button='OK')

In [ ]:
#getText = pymsgbox.confirm(text='Choose OK or Cancel', title='', buttons=['OK', 'Cancel'])

In [ ]:
#print(getText)

In [ ]:
import psycopg2

In [ ]:
conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")

In [ ]:
# def dropPlayerTable():
#     conn = None
#     try:
#         conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
#         cur = conn.cursor()
#         sql = """DROP TABLE IF EXISTS player ; """
#         cur.execute(sql)
#         conn.commit()
#     except(Exception, psycopg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()

In [ ]:
#dropPlayerTable()

In [ ]:
# def createPlayerTable():
#     conn = None
#     try:
#         conn = psycopg2.connect("dbname='mydatabase2' user='myuser' host='localhost' port='65432' password='123qwe'")
#         cur = conn.cursor()
#         sql = """
# CREATE TABLE player (
#     Date DATE,           --01
#     Num int,             --02
#     Nat VARCHAR(30),     --03
#     Player VARCHAR(30),  --04
#     PlayerID BIGINT,     --05
#     Spacial VARCHAR(30), --06
#     st VARCHAR(10),      --07  Status Yellow, Red
#     Age VARCHAR(10),     --08
#     Since VARCHAR(30),   --09
#     TSI BIGINT,          --10
#     LS INT,              --11
#     XP INT,              --12
#     Fo INT,              --13
#     Stm INT,             --14
#     Lo INT,              --15
#     MB boolean,          --16
#     KP INT,              --17
#     DF INT,              --18
#     PM INT,              --19
#     WI INT,              --20
#     PS INT,              --21
#     SC INT,              --22
#     SP INT,              --23
#     con BIGINT,          --24 Psico
#     Last DATE,           --25
#     RT NUMERIC(2, 1),    --26 Rating
#     Po VARCHAR(5),       --27 Position
#     Wage BIGINT,         --28
#     G INT,               --29
#     --TC VARCHAR(1),     --
#     --PH VARCHAR(1),     --
#     KP_P NUMERIC(4, 2),  --30 KP Position
#     WB_P NUMERIC(4, 2),  --31 WB Position
#     CD_P NUMERIC(4, 2),  --32 CD Position
#     W_P NUMERIC(4, 2),   --33 W  Position
#     IM_P NUMERIC(4, 2),  --34 IM Position
#     FW_P NUMERIC(4, 2)   --35 FW Position
# ) ; """
#         cur.execute(sql)
#         conn.commit()
#     except(Exception, psycopg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()

In [ ]:
# createPlayerTable()

In [ ]:
ht_player_pg = HattrickPlayerPostgreSQL()

In [ ]:
ht_player_pg.select_count_player(conn)

In [ ]:
import csv
with open(folder + '/player.txt', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        if row[0] != "Number" :
            ht_player_pg.insert_player(conn, folder.replace('/', '-'), row)

In [ ]:
ht_player_pg.select_count_player(conn)

In [ ]:
tuple_list = ht_player_pg.select_player(conn, folder)
ht_player_pg.print(tuple_list)

In [ ]:
conn.close()
conn

In [ ]:
conn = None
conn